# -Introduction
l'agence de stante publique France a lance un apel a projets pour trouver
des idees innovantes d'applicationq en lien avec l'alimentation
Nous avons decidé de nous attaquer a l'obésite.
Nous voulons etudier la possiblite qu'une application informe, pour n'importe quel produit,
le consommation du caractere Nutriscore.
La prediction du Nutriscore doit etre realise a partir de la liste des nutiments du produit.
Le Nutriscore est une classification des produits alimentaires, les classes sont nommées par ordre
de qualite nutritionnelle de A a E


# Importation des données 

In [1]:
import dask.dataframe as dd
import numpy as np
import pandas as pd
import re
import IPython.display
import matplotlib.pyplot as plt 
import seaborn as sns
plt.style.use("seaborn")
%matplotlib inline
dtypes = {
    "brand_owner":object,
    "generic_name":object,
    "ingredients_from_palm_oil_tags":object,
    "ingredients_that_may_be_from_palm_oil_tags":object,
    "origins":object,
    "origins_en":object,
    "origins_tags":object,
    "packaging_text":object,
    "traces":object,
    "traces_en":object,
    "traces_tags":object,
    "code":object,
    "abbreviated_product_name": object
    }
#data = dd.read_csv("data.csv", sep = "\t", dtype=dtypes, low_memory=False).compute()


In [2]:
url = r'https://raw.githubusercontent.com/Loridan/openclassrooms-iml-projects/main/project2/en.openfoodfacts.org.products-df5.csv'


NameError: name 'data' is not defined

#  Analyse des variables
A ce stade nous cherchons à déterminer les variables qui sont à la fois :

    -- perinentes dans le cadre de l'exploration
    -- discriminantes pour catégoriser les observations et dans un second temps
    -- qui ont un taux de complétion élevé.
Concernant le dernier point, Il est acceptable de considérer des observations qui ont certaines variables non renseignées sous deux conditions:

Que pour l'observation la proportion de variables retenues non renseignées soit faible
Que globalement le taux de complétion global de chaque variable reste élevé
Dans ces deux cas il sera possible d'envisager des stratégies d'imputation pour compléter les valeurs manquantes.

Nous commençons par afficher l'ensemble des variables